In [1]:
from __future__ import print_function
from functools import reduce


import SimpleITK as sitk
import sys
import os


def command_iteration(method) :
    print("{0:3} = {1:7.5f} : {2}".format(method.GetOptimizerIteration(),
                                           method.GetMetricValue(),
                                           method.GetOptimizerPosition()))


pixelType = sitk.sitkFloat32

fixed = sitk.ReadImage('t1.jpg', sitk.sitkFloat32)
fixed = sitk.Normalize(fixed)
fixed = sitk.DiscreteGaussian(fixed, 2.0)


moving = sitk.ReadImage('t2.jpg', sitk.sitkFloat32)
moving = sitk.Normalize(moving)
moving = sitk.DiscreteGaussian(moving, 2.0)


R = sitk.ImageRegistrationMethod()

R.SetMetricAsJointHistogramMutualInformation()

R.SetOptimizerAsGradientDescentLineSearch(learningRate=1.0,
                                          numberOfIterations=200,
                                          convergenceMinimumValue=1e-5,
                                          convergenceWindowSize=5)

R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))

R.SetInterpolator(sitk.sitkLinear)

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print(" Iteration: {0}".format(R.GetOptimizerIteration()))
print(" Metric value: {0}".format(R.GetMetricValue()))

  0 = -0.76994 : (-5.237794887425869e-05, 6.375517356673184e-05)
  1 = -0.76994 : (-0.00013374406000330206, 0.00016279498368574767)
  2 = -0.76994 : (-0.0003864089470039541, 0.00047033560695084945)
  3 = -0.76994 : (-0.0011095943545940093, 0.0013505719861829463)
  4 = -0.76994 : (-0.002062857333406321, 0.00251072701429781)
-------
itk::simple::Transform
 TranslationTransform (0x1692e30)
   RTTI typeinfo:   itk::TranslationTransform<double, 2u>
   Reference Count: 2
   Modified Time: 26421
   Debug: Off
   Object Name: 
   Observers: 
     none
   Offset: [-0.000133744, 0.000162795]

Optimizer stop condition: GradientDescentLineSearchOptimizerv4Template: Convergence checker passed at iteration 5.
 Iteration: 5
 Metric value: -0.7699392704419347


In [9]:

import numpy as np

if ( not "SITK_NOSHOW" in os.environ ):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(1)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)

    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)

    out_as_numpy = sitk.GetArrayFromImage(out)
    print(out_as_numpy)
    import png
    png.from_array(out_as_numpy, 'L').save("small_smiley.png")

    import matplotlib.pyplot as plt
    plt.imshow(out_as_numpy)

  #  sitk.WriteImage(out, 'out.png')
   # sitk.Show( cimg, "ImageRegistration2 Composition" )

TypeError: in method 'ResampleImageFilter_SetReferenceImage', argument 2 of type 'itk::simple::Image const &'